In [ ]:
# -*- coding: utf-8 -*-
"""TP Final con 3 valores con base flanqueada y sin NaN - Posible que usemos este.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1uPPbibvevdYszz1qkPdq7V-GnnCrQee3

#Clasificación Estelar

##Desde siempre hemos mirado al cielo pensando en ¿quienes sómos? ¿por qué estamos aquí? ¿Qué son las estrellas?, en base a estas preguntas y otras mas profundas y específicas, nace el estudio de los astros, la astronomía.

##En la Astronomía, la clasificación estelar se basa en características espectrales. Algunas de estas características son: rotación, temperatura, velocidad radial, distancia, etc.

##En este data frame

##A partir de unas serie de condiciones observadas en el espacio, se puede determinar si un cuerpo es una galaxia, una estrella o un cuásar?

### Nota: Los objetos cuasi estelares (quasar - QSO) son una clase de objetos que se encuentran más allá de nuestra galaxia, la Vía Láctea, que tienen una apariencia visual similar a la de una estrella, excepto que el espectro óptico tiene un gran corrimiento al rojo ("redshift").

## Acá viene la explicación super cheta de Rodri

#  Falta definir que hay en cada colmuna

    # 'obj_ID':       'Object Identifier',
    # 'alpha':        'Right Ascension',
    # 'delta':        'Declination',
    # 'u':        'Ultraviolet filter',
    # 'g':        'Gren filter',
    # 'r':        'Red Filter',
    # 'i':        'Near Infrared filter',
    # 'z':        'Infrared filter',
    # 'run_ID':       'Run Number(Scan ID)',
    # 'rerun_ID':       'Rerun Number',Especifica cómo fue procesada la imagen
    # 'cam_col':        'Camera column(Scanline ID)',
    # 'field_ID':       'Field ID',
    # 'spec_obj_ID':        '****',
    # 'class':        'Object class',
    # 'redshift':       '****',
    # 'plate':        'Plate ID****',
    # 'MJD':        'Modified Julian Datre****',
    # 'fiber_ID':       'Fiber ID'
"""

# Cargamos las librerías con las que vamos a trabajar

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Declaramos la base de datos y la volvemos "df" para poder manipularla

df = pd.read_csv('https://raw.githubusercontent.com/MKSuser/Unsam1/refs/heads/main/TP%20Mate3/star_classification.csv')

# Obtenemos 5 muestras para poder observar como están compuestas

df.sample(5)

# Podemos observar con mejor detalle el tipo de las variables que ocupan cada columna

df.info()

# Sabiendo que lo que buscamos es poder determinar qué es lo que observamos, obtenemos qué clases (output) hay,
# y cuántos resultados hay de cada una.

df["class"].value_counts()

# Por si llegan a haber muestras sin datos, los limpiamos. Si no hay NaN no va a pasar nada.

df = df.dropna()

df["class"].value_counts()

# Contamos con la misma cantiadad de resultados, por lo que no habría ningún NaN.

# Entre los datos de la base, nos encontramos con que el output se determina con las palabras "GALAXY" (galaxia), "STAR" (estrella) y "QSO" (Objeto cuasi estelar).
# Para poder buscar correlatividades, convertimos estas palabras en los valores 0, 1 y 2.
# Generamos un diccinario para poder reemp los datos

class_rep = {
     "GALAXY": 0,
     "STAR":   1,
     "QSO":    2,
}

# Generamos un nuevo df para poder hacer pruebas

df_limpio = df

# Reemplazamos los datos del diccionario "class_rep" por los que figuran en la colmuna "class"

df_limpio['class'] = df_limpio['class'].replace(class_rep)

# Mover la columna 'class' al final
df_limpio = df_limpio[[col for col in df.columns if col != 'class'] + ['class']]

# Asi como reemplazamos eso, podemos también sacar colmunas con "df = df.drop ('nombreColumna', axis = 1)"
# df = df.drop ('rerun_ID', axis = 1)

# Verifiamos que en la nueva base tenemos valores numericos en la columna "class" en lugar de texto

df_limpio.sample(10)

# Imprimimos los histogramas de cada columna para poder evaluar distribución de datos.

df_limpio.hist(figsize=(15, 15))

# Como observamos datos variados optamos por trabajarlo por cuartiles

df_limpio_salida = df_limpio['class']
df_limpio_red = df_limpio['redshift']

# Primero separamos los cuartiles teniendo encuenta antes del 25%, y luego del 75%
Q1 = df_limpio.quantile(0.25)
Q3 = df_limpio.quantile(0.75)

# Tomamos la diferencia para poder laburar
IQR = Q3 - Q1

# Identificar valores atípicos utilizando el criterio del IQR
atipicos = ((df_limpio < (Q1 - 1.5 * IQR)) | (df_limpio > (Q3 + 1.5 * IQR)))

# Con esto se pueden saber los valores
num_atipicos = atipicos.sum()
print("num_atipicos:\n", num_atipicos)

# Saco del df_limpio las filas con valores atípicos
df_limpio = df_limpio.where(np.invert(atipicos))

# Insertamos la columna de salida original
df_limpio['class'] = df_limpio_salida
df_limpio['redshift'] = df_limpio_red

# Nuevos histogramas
df_limpio.hist(figsize=(15, 15))

# # Observamos que 'rerun_ID' repite datos por lo que lo sacamos del df2 para que se use innecesariamente en los cálculos
df_limpio = df_limpio.drop ('rerun_ID', axis = 1)

df_limpio.describe()

# Emitimos una estadística rápida de los datos pero normalizados

df_limpio_salida = df_limpio['class']

df_limpio_stats = df_limpio.describe().T
df_limpio_n = (df_limpio - df_limpio_stats['mean']) / df_limpio_stats['std']  # Para normalizar: (valor - promedio) / desv_estandar

df_limpio_n['class'] = df_limpio_salida

df_limpio_n.describe()

df_limpio = df_limpio.dropna()
# df.to_csv('prueba.csv')

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df_limpio.corr(), annot=True, linewidths=0.5, fmt= '.2f',ax=ax)
ax.set_xticklabels(df_limpio.columns.to_list(), rotation=45, ha='right')
plt.title("Matriz de Correlaciones")
plt.show()

# Concluimos que hay una correlacion entre el redshit y nuestra valor de salida "class"

# Para que se pueda trabajar con valores correlativos optamos por trabajar con la base como llega.
# Por lo que vemos los valores dan más acorde.

# Limpiamos las colmunas con poca correlacion

df_limpio = df_limpio.drop ('obj_ID', axis = 1)
df_limpio = df_limpio.drop ('alpha', axis = 1)
df_limpio = df_limpio.drop ('delta', axis = 1)
# df_limpio = df_limpio.drop ('u', axis = 1)
df_limpio = df_limpio.drop ('g', axis = 1)
# df_limpio = df_limpio.drop ('r', axis = 1)
# df_limpio = df_limpio.drop ('i', axis = 1)
# df_limpio = df_limpio.drop ('z', axis = 1)
df_limpio = df_limpio.drop ('run_ID', axis = 1)
# df_limpio = df_limpio.drop ('rerun_ID', axis = 1)
df_limpio = df_limpio.drop ('cam_col', axis = 1)
df_limpio = df_limpio.drop ('field_ID', axis = 1)
# df_limpio = df_limpio.drop ('spec_obj_ID', axis = 1)
# df_limpio = df_limpio.drop ('plate', axis = 1)
# df_limpio = df_limpio.drop ('MJD', axis = 1)
df_limpio = df_limpio.drop ('fiber_ID', axis = 1)

fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df_limpio.corr(), annot=True, linewidths=0.5, fmt= '.2f',ax=ax)
ax.set_xticklabels(df_limpio.columns.to_list(), rotation=45, ha='right')
plt.title("Matriz de Correlaciones")
plt.show()
